# Imports # 

In [67]:
import pandas as pd
import glob
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

# Read in all data into a pandas DataFrame # 

In [68]:
dfs = []

for file in glob.glob("*.csv"):
    dfs.append(pd.read_csv(file))
    print(file)
    
df = pd.concat(dfs)

df = df.reset_index()
df.columns = df.iloc[0]
df = df.iloc[1:]
df = df.set_index("Date")
#df = pd.read_csv("Poloniex_DASHBTC_1h.csv")

Poloniex_DASHBTC_1h.csv
Poloniex_ETCBTC_1h.csv
Poloniex_ETHBTC_1h.csv
Poloniex_LSKBTC_1h.csv
Poloniex_LTCBTC_1h.csv
Poloniex_NXTBTC_1h.csv
Poloniex_POTBTC_1h.csv
Poloniex_XEMBTC_1h.csv
Poloniex_XMRBTC_1h.csv
Poloniex_XRPBTC_1h.csv


In [69]:
#Only keep the columns we need
df = df[['Symbol','High','Low','Close']]

# Generate 3D tensors for input into the neural net # 

In [126]:
#Create 3D tensor
def generate_3D_tensor(df, time_index):
    
    #Collect only the first 50 times 
    relevant_df = df.loc[time_index]
    high_df = relevant_df[["Symbol","High"]]
    low_df = relevant_df[["Symbol","Low"]]
    close_df = relevant_df[["Symbol","Close"]]

    #Generate the dataframes in the shape we need
    high_df = high_df.pivot(columns='Symbol', values='High').T
    low_df = low_df.pivot(columns='Symbol', values='Low').T
    close_df = close_df.pivot(columns='Symbol', values='Close').T

    #Generate 3D Tensor
    high = high_df.as_matrix()
    low = low_df.as_matrix()
    close = close_df.as_matrix()

    full = np.stack([high,low,close],axis=0)
    full = full.astype('float64')
    full = torch.from_numpy(full)
    
    return full

In [83]:
#Get a Series of all times within the dataframe
full_times_index = df.index.drop_duplicates().sort_values()

x_train = []

for i in range(len(full_times_index)-50):
    x_train.append(generate_3D_tensor(df,full_times_index[i:i+50]))
    if (i%1000) == 0:
        print("Completed another 10%!")



Completed another 10%!
Completed another 10%!
Completed another 10%!
Completed another 10%!
Completed another 10%!
Completed another 10%!
Completed another 10%!
Completed another 10%!
Completed another 10%!
Completed another 10%!


In [100]:
%%timeit

# Time and profile the generate_3D_tensor function
generate_3D_tensor(df,full_times_index[:50])

13.8 ms ± 134 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [101]:
# Stack the generated 3D tensors and save it to a pickle file to reduce generating times in the future
x_train_stacked = torch.stack(x_train)
x_train_stacked.size()

import pickle
pickle.dump( x_train_stacked, open( "x_train.p", "wb" ) )

# Define the Network #

In [51]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 4 * 4, 120)     #Remember to change this based on the input layer dimensions!
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)


Date,2017-07-01 01-PM,2017-07-01 02-PM,2017-07-01 03-PM,2017-07-01 04-PM,2017-07-01 05-PM,2017-07-01 06-PM,2017-07-01 07-PM,2017-07-01 08-PM,2017-07-01 09-PM,2017-07-01 10-PM,...,2017-07-03 02-PM,2017-07-03 03-AM,2017-07-03 03-PM,2017-07-03 04-AM,2017-07-03 04-PM,2017-07-03 05-AM,2017-07-03 05-PM,2017-07-03 06-AM,2017-07-03 06-PM,2017-07-03 07-AM
Symbol,,,,,,,,,,,,,,,,,,,,,
DASHBTC,0.07075,0.07051,0.07095,0.07089,0.06993,0.0697,0.06981,0.0697,0.06923,0.06814,...,0.07047,0.07003,0.0706,0.06983,0.07056,0.07016,0.07047,0.07004,0.0705,0.07
ETCBTC,0.00701,0.006967,0.006995,0.00698,0.006923,0.006899,0.006825,0.006815,0.006825,0.006826,...,0.00689,0.006898,0.006891,0.00694,0.006868,0.006979,0.006834,0.006908,0.00687,0.00689
ETHBTC,0.1107,0.1097,0.1109,0.1108,0.1096,0.1088,0.1073,0.1082,0.1074,0.109,...,0.1109,0.1135,0.11,0.1126,0.1104,0.1136,0.111,0.1126,0.1112,0.1121
LSKBTC,0.0009901,0.0009847,0.001004,0.001004,0.000984,0.0009633,0.0009761,0.0009761,0.0009661,0.0009745,...,0.001017,0.001021,0.0009973,0.001006,0.000997,0.0009885,0.0009857,0.0009911,0.00098,0.0009861
LTCBTC,0.01588,0.01587,0.0159,0.01594,0.01594,0.01585,0.0157,0.01568,0.01566,0.01565,...,0.01844,0.01672,0.01832,0.01769,0.01862,0.018,0.01852,0.0182,0.01849,0.01805
NXTBTC,6.23e-05,6.216e-05,6.093e-05,5.907e-05,6.02e-05,5.829e-05,5.723e-05,5.573e-05,5.52e-05,5.651e-05,...,5.549e-05,5.431e-05,5.465e-05,5.449e-05,5.475e-05,5.479e-05,5.45e-05,5.454e-05,5.476e-05,5.381e-05
POTBTC,3.939e-05,3.93e-05,3.952e-05,3.905e-05,3.865e-05,3.822e-05,3.721e-05,3.642e-05,3.697e-05,3.676e-05,...,3.805e-05,3.866e-05,3.805e-05,3.867e-05,3.836e-05,3.872e-05,3.815e-05,3.835e-05,3.844e-05,3.756e-05
XEMBTC,6.35e-05,6.349e-05,6.265e-05,6.263e-05,6.264e-05,6.228e-05,6.166e-05,6.11e-05,6.088e-05,6.11e-05,...,6.409e-05,6.44e-05,6.407e-05,6.411e-05,6.588e-05,6.368e-05,6.6e-05,6.459e-05,6.648e-05,6.42e-05
XMRBTC,0.01743,0.01738,0.01732,0.01725,0.01721,0.01717,0.01715,0.01695,0.01703,0.01715,...,0.01717,0.01702,0.01717,0.0172,0.01726,0.0174,0.01735,0.0174,0.01743,0.0171
